In [ ]:
import matplotlib.pyplot as plt #library for visualizing data
%matplotlib widget 
#setting for jupyter lab
plt.rcParams['figure.figsize'] = [10, 5] #setting figure size (plots)
from matplotlib import cm #colormap for plots


import pandas as pd  # (software library for data analysis and manipulation, https://pandas.pydata.org/docs/)
import numpy as np  # (software library for matrix multiplications, https://numpy.org/doc/)
from numpy.random import randint #function for random integers
from numpy.random import rand #function for random floats

import statistics as stats  # (python module for statistic calculations, https://docs.python.org/3/library/statistics.html)
import time #python time module
import math #python module for mathematical operations

In [ ]:
def decode(bitstring, n_bits, ranges):
    '''decode bitstring to numbers'''
    decoded = list()
    largest = 2**n_bits
    for i in range(len(ranges)):
        
        # extract the substring
        start, end = i * n_bits, (i * n_bits)+n_bits
        substring = bitstring[start:end]
        
        chars = ''.join([str(s) for s in substring]) # convert bitstring to a string of chars
        integer = int(chars, 2) #convert string to int
        
        value = ranges[i][0] + (integer/largest) * (ranges[i][1] - ranges[i][0])
        
        decoded.append(value) #store
        
    return decoded
 
def selection(pop, scores, opponents=2):
    '''select individuals for reproduction selection'''
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), opponents):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]
 
    
def crossover(p1, p2, co_rate):
    '''crossover two parents to create two children'''
    c1, c2 = p1.copy(), p2.copy() # children are copies of parents by default

    # check for recombination
    if rand() < co_rate:
        pt = randint(1, len(p1)-2) # select crossover point that is not on the end of the string
        
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]
 
    
def mutation(bitstring, mut_rate):
    '''apply mutation to bitsring'''
    bitstring = bitstring.copy()
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < mut_rate:
            bitstring[i] = 1 - bitstring[i] # flip the bit
    return bitstring


In [ ]:
# problem 1

def fitness(x):
    '''fitness function to determine quality of solutions'''
    # des_result = 0
    # fitness = abs(des_result - (x[0]-10)**2.0 + math.log(x[1]+2) + math.sin(x[2]) - (3*x[3]+1))
    
    result = (1 - x[0]/2 + x[0]**6 + x[1]**3) * -np.exp(-x[0]**2 - x[1]**2) + 1.7
    return result

#abs(des_result - (x[0]-10)**2.0 + math.log(x[1]+2) + math.sin(x[2]) - (3*x[3]+1))
#ranges = [[-10000, 10000],[0.00001, 10000],[-10000, 10000],[-10000, 10000]] #ranges for the variables

ranges = [[-6,6],[-6,6]]

n_bits = 12 #bits per variable

n_generations = 100 #number of generations (iterations)
n_population = 200 #population size

co_rate = 0.6 # crossover rate
mut_rate =  0.02#1 / (float(n_bits) * len(ranges)) # mutation rate

min_std_dev = 0.02

In [ ]:
# problem 2
def fitness(x):
    '''fitness function to determine quality of solutions'''
    des_result = 0
    result = (x[0]-10)**2.0 + math.log(x[1]+2) + math.sin(x[2]) - (3*x[3]+1)
    
    fitness = abs(des_result - result)
    return fitness

ranges = [[-10000, 10000],[-1.999, 10000],[-10000, 10000],[-10000, 10000]] #ranges for the variables


n_bits = 16 #bits per variable

n_generations = 200 #number of generations (iterations)
n_population = 300 #population size

co_rate = 0.4 # crossover rate
mut_rate =  0.03#1 / (float(n_bits) * len(ranges)) # mutation rate

min_std_dev = 0.01

In [ ]:
population = []

for i in range(n_population):
    chromosom = randint(0, 2, n_bits*len(ranges)).tolist()
    population.append(chromosom)
    
best, best_eval = None, float('inf')

population_df = pd.DataFrame(population)
    
display(population_df)

In [ ]:
t_0 = time.perf_counter()

history = []
no_div = 0

for gen in range(n_generations):
        decoded = [decode(p, n_bits, ranges) for p in population] # decode population
        scores = [fitness(d) for d in decoded] # evaluate all candidates in the population
        
        # check for new best solution
        for i in range(n_population):
            if scores[i] < best_eval:
                best, best_eval = population[i], scores[i]
                print(f'''>{gen}< New best fit: f({decoded[i]}) = {scores[i]}''')
        
        
        ## select best parents by sorting
        # population_score_df = pd.DataFrame([population, scores], index=['pop','score']).T.sort_values(by='score')
        # parents = population_score_df.head(int(len(population)/2))['pop'].tolist()
        
        
        # select parents by tournament approach
        parents = []
        for i in range(n_population):
            parents.append(selection(population, scores))
                        
        # create the next generation of offspring
        children = list()
        for i in range(0, len(parents), 2):

            p1, p2 = parents[i], parents[i+1] # get selected parents in pairs
            
            # crossover and mutation
            for c in crossover(p1, p2, co_rate):
                c = mutation(c, mut_rate) # mutation
                children.append(c) # store for next generation

        population = population + children # expand population
        
        decoded = [decode(p, n_bits, ranges) for p in population] # decode new population
        scores = [fitness(d) for d in decoded] # evaluate all candidates in the population
        
        population_score_df = pd.DataFrame([population, scores], index=['pop','score']).T.sort_values(by='score') #sort population
        population = population_score_df.head(n_population)['pop'].tolist() #select best individuals to stay in population
        
        population_df = pd.DataFrame(population)
        pop_std_dev = population_df.std(axis=0).mean()
        
        history.append([gen, best_eval, stats.mean(scores), stats.median(scores)]) #store history

        
        if pop_std_dev < min_std_dev:
            print(f'>{gen}< Diversity at {pop_std_dev} and below threshold. Aborting...')
            break


        
t_1 = time.perf_counter()

        
print('Finished.\n')
decoded = decode(best, n_bits, ranges)
print(f'''Best fit: f({decoded}) = {best_eval}''')
print(f'Time consumed: {round(t_1-t_0,3)}')



In [ ]:
history_pd = pd.DataFrame(history, columns = ['gen', 'fittest', 'mean_fit', 'median_fit']).set_index('gen')

plt.close('all')
history_pd['fittest'].plot(title='Fitness over generations', ylim=[0, history_pd['fittest'].max()*1.1])

In [ ]:
plt.close('all')
history_pd[['mean_fit', 'median_fit']].plot(title = 'Mean and median fitness over generations', ylim=[0, history_pd['mean_fit'].max()*1.1])